In [1]:
import os
import sys
import operator

import pandas as pd
import numpy as np

from langdetect import detect

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data_input.load_data import load_corpus

import readability
import spacy

In [2]:
# load the data for the three evaluation cycles
corpus1 = load_corpus("../../data/data_cycle1")

In [3]:
corpus1.shape

(32387, 14)

In [4]:
# filter German language arguments
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

def detect_languages_in_dataset(argu_corpus):
    argu_corpus['language'] = argu_corpus['argument'].apply(detect_language)
    return argu_corpus

result_langu_df = detect_languages_in_dataset(corpus1)
corpus_de = result_langu_df[result_langu_df['language'] == 'de']
corpus_de = corpus_de.reset_index(drop=True)

In [5]:
corpus_de.shape

(22353, 15)

In [6]:
corpus_de

,argument_id,argument,stance,topic,gender,age,residence,civil_status,denomination,education,political_spectrum,important_political_issues,rile,galtan,language
0,201900,Das Schweizer Volk hat die MEI angenommen und ...,FAVOR,Immigration,Männlich,18-34,Land,Ledig,Christ-katholisch,Fachhochschule,Mitte und Konservativ-Liberal,"[Liberale Wirtschaftspolitik, Restriktive Migr...",Mitte,Konservativ-Liberal,de
1,201901,Eine Legalisierung von Cannabis entlasten die ...,FAVOR,Society,Männlich,18-34,Land,Ledig,Christ-katholisch,Fachhochschule,Mitte und Konservativ-Liberal,"[Liberale Wirtschaftspolitik, Restriktive Migr...",Mitte,Konservativ-Liberal,de
2,201902,Durch die Förderung der familienergänzenden Be...,FAVOR,Welfare,Weiblich,35-49,Land,Ledig,Nicht bekannt,Universität,Mitte und Konservativ,"[Offene Aussenpolitik, Liberale Wirtschaftspol...",Mitte,Konservativ,de
3,201903,Ich ziehe eine Elternzeit vor. Die Zeit nach d...,AGAINST,Welfare,Weiblich,35-49,Land,Ledig,Nicht bekannt,Universität,Mitte und Konservativ,"[Offene Aussenpolitik, Liberale Wirtschaftspol...",Mitte,Konservativ,de
4,201904,Unser Asylrecht muss konsequent angewendet wer...,AGAINST,Immigration,Weiblich,35-49,Land,Ledig,Nicht bekannt,Universität,Mitte und Konservativ,"[Offene Aussenpolitik, Liberale Wirtschaftspol...",Mitte,Konservativ,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22348,2019031140,Das Stimmrechtsalter ab 16 könnte prinzipiell ...,FAVOR,Political System,Weiblich,35-49,Land,Verheiratet,Evangelischreformiert/protestantisch,Universität,Links und Konservativ-Liberal,"[Ausgebauter Umweltschutz, Ausgebauter Sozials...",Links,Konservativ-Liberal,de
22349,2019031147,Die heutige Technologie erlangt eigentlich E-V...,FAVOR,Political System,Weiblich,50-64,Land,Ledig,Römisch-katholisch,Universität,Mitte und Konservativ-Liberal,"[Ausgebauter Umweltschutz, Ausgebauter Sozials...",Mitte,Konservativ-Liberal,de
22350,2019031148,Nur Volljaehrige sollen weiter stimmrechtig bl...,AGAINST,Political System,Weiblich,50-64,Land,Ledig,Römisch-katholisch,Universität,Mitte und Konservativ-Liberal,"[Ausgebauter Umweltschutz, Ausgebauter Sozials...",Mitte,Konservativ-Liberal,de
22351,2019031162,La transparence est une vertue.\n\nTransparenz...,FAVOR,Political System,Männlich,35-49,Land,Verheiratet,Römisch-katholisch,Nicht bekannt,Links und Konservativ-Liberal,"[Offene Aussenpolitik, Law & Order, Ausgebaute...",Links,Konservativ-Liberal,de


# Preprocessing
#### Clean data to improve meaningfulness of results

In [7]:
ind_variables1 = ["gender", "age", "residence", "civil_status", "denomination", "education", 
                  "rile", "galtan", "important_political_issues"]
ind_variables2 = ["stance", "topic"]

In [8]:
# check sparsity
[corpus_de[col].value_counts() for col in ind_variables1]

[gender
 Männlich    14308
 Weiblich     8045
 Name: count, dtype: int64,
 age
 18-34    7467
 50-64    7215
 35-49    6393
 65+      1278
 Name: count, dtype: int64,
 residence
 Land     20257
 Stadt     2096
 Name: count, dtype: int64,
 civil_status
 Verheiratet                   8190
 Nicht bekannt                 7566
 Ledig                         5294
 Geschieden                     575
 Konkubinat                     395
 Verwitwet                      234
 Eingetragene Partnerschaft      52
 Getrennt                        46
 Aufgelöste Partnerschaft         1
 Name: count, dtype: int64,
 denomination
 Nicht bekannt                             9018
 Evangelischreformiert/protestantisch      5484
 Römisch-katholisch                        4221
 Konfessionslos                            2993
 Andere christliche Gemeinschaften          506
 Christ-katholisch                           69
 Christlich-orthodox                         22
 Andere Kirchen/Religionsgemeinschaften      1

In [9]:
# Summarize groups
corpus_de.loc[corpus_de["education"] == "Höhere Fachschule", 
              "education"] = "Höhere Berufsausbildung"
corpus_de.loc[corpus_de["education"] == "Handelsschule", 
              "education"] = "Höhere Berufsausbildung"
corpus_de.loc[corpus_de["education"] == "Berufsmatura", 
              "education"] = "Berufsmatura/Diplommittelschule"
corpus_de.loc[corpus_de["education"] == "Diplommittelschule", 
              "education"] = "Berufsmatura/Diplommittelschule"

In [11]:
# Drop groups with rare occurences (<50)
corpus_de = corpus_de.drop(corpus_de[
    corpus_de["civil_status"].isin(["Getrennt"])].index)
corpus_de = corpus_de.drop(corpus_de[
    corpus_de["civil_status"].isin(["Aufgelöste Partnerschaft"])].index)

corpus_de = corpus_de.drop(corpus_de[
    corpus_de["denomination"].isin(["Christlich-orthodox"])].index)
corpus_de = corpus_de.drop(corpus_de[
    corpus_de["denomination"].isin(["Andere Kirchen/Religionsgemeinschaften"])].index)
corpus_de = corpus_de.drop(corpus_de[
    corpus_de["denomination"].isin(["Jüdische Gemeinschaften"])].index)
corpus_de = corpus_de.drop(corpus_de[
    corpus_de["denomination"].isin(["Islamische Gemeinschaften"])].index)

In [12]:
# Drop rows with missing values
corpus_de = corpus_de.drop(corpus_de[corpus_de["civil_status"].isin(["Nicht bekannt"])].index)
corpus_de = corpus_de.drop(corpus_de[corpus_de["denomination"].isin(["Nicht bekannt"])].index)
corpus_de = corpus_de.drop(corpus_de[corpus_de["education"].isin(["Nicht bekannt"])].index)

In [13]:
[corpus_de[col].value_counts() for col in ind_variables1]

[gender
 Männlich    7474
 Weiblich    3815
 Name: count, dtype: int64,
 age
 50-64    3830
 35-49    3563
 18-34    3320
 65+       576
 Name: count, dtype: int64,
 residence
 Land     10301
 Stadt      988
 Name: count, dtype: int64,
 civil_status
 Verheiratet                   6417
 Ledig                         3904
 Geschieden                     417
 Konkubinat                     369
 Verwitwet                      130
 Eingetragene Partnerschaft      52
 Name: count, dtype: int64,
 denomination
 Evangelischreformiert/protestantisch    4801
 Römisch-katholisch                      3438
 Konfessionslos                          2496
 Andere christliche Gemeinschaften        490
 Christ-katholisch                         64
 Name: count, dtype: int64,
 education
 Universität                        4903
 Fachhochschule                     2007
 Höhere Berufsausbildung            1488
 Berufslehre/Berufsschule           1288
 Gymnasium/Seminar                   874
 Berufsmatura/Dipl

In [14]:
corpus_de.shape

(11289, 15)

# Feature Extraction

### Surface features

In [15]:
corpus_de['FleschReadingEase'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['readability grades']['FleschReadingEase'])
corpus_de['GunningFogIndex'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['readability grades']['GunningFogIndex'])

corpus_de['characters_per_word'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['sentence info']['characters_per_word'])
corpus_de['words_per_sentence'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['sentence info']['words_per_sentence'])
corpus_de['type_token_ratio'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['sentence info']['type_token_ratio'])
corpus_de['long_words'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['sentence info']['long_words'])
corpus_de['complex_words'] = corpus_de['argument'].apply(lambda x: readability.getmeasures(x, lang='de')['sentence info']['complex_words'])

In [16]:
corpus_de.head()

,argument_id,argument,stance,topic,gender,age,residence,civil_status,denomination,education,...,rile,galtan,language,FleschReadingEase,GunningFogIndex,characters_per_word,words_per_sentence,type_token_ratio,long_words,complex_words
0,201900,Das Schweizer Volk hat die MEI angenommen und ...,FAVOR,Immigration,Männlich,18-34,Land,Ledig,Christ-katholisch,Fachhochschule,...,Mitte,Konservativ-Liberal,de,25.703448,15.737931,5.758621,29.0,0.896552,9,3
1,201901,Eine Legalisierung von Cannabis entlasten die ...,FAVOR,Society,Männlich,18-34,Land,Ledig,Christ-katholisch,Fachhochschule,...,Mitte,Konservativ-Liberal,de,7.991154,13.476923,6.576923,26.0,0.923077,11,2
33,2019054,Die jetzige finanzielle Situation der Pensions...,FAVOR,Welfare,Männlich,50-64,Land,Verheiratet,Evangelischreformiert/protestantisch,Höhere Berufsausbildung,...,Mitte,Konservativ-Liberal,de,26.470000,8.036364,6.545455,11.0,1.000000,6,1
34,2019055,"Dort wo die Leistungen nicht angebracht sind, ...",FAVOR,Welfare,Männlich,50-64,Land,Verheiratet,Evangelischreformiert/protestantisch,Höhere Berufsausbildung,...,Mitte,Konservativ-Liberal,de,44.405000,12.000000,6.000000,10.0,1.000000,4,2
35,2019056,"Wünschenswert, aber eine Herkulesaufgabe für d...",FAVOR,Education,Männlich,50-64,Land,Verheiratet,Evangelischreformiert/protestantisch,Höhere Berufsausbildung,...,Mitte,Konservativ-Liberal,de,-34.129231,11.353846,7.923077,13.0,0.923077,7,2


#### Mean and standard deviation for the features

In [17]:
surface_features = ['FleschReadingEase', 'GunningFogIndex', 'characters_per_word', 
                             'words_per_sentence', 'type_token_ratio', 'long_words', 'complex_words']

In [18]:
for feat in surface_features:
    print(feat, round(corpus_de[feat].mean(),2), round(corpus_de[feat].min(),2), 
          round(corpus_de[feat].max(),2), round(corpus_de[feat].std(),2))

FleschReadingEase 19.15 -177.92 103.7 27.65
GunningFogIndex 14.33 1.4 36.4 6.23
characters_per_word 6.41 3.75 14.25 0.96
words_per_sentence 23.36 3.5 85.0 14.26
type_token_ratio 0.95 0.64 1.0 0.06
long_words 8.89 0 32 5.63
complex_words 2.95 0 15 2.28


### Syntactic features

In [19]:
#ADJ: adjective
#ADP: adposition
#ADV: adverb
#AUX: auxiliary
#CCONJ: coordinating conjunction
#DET: determiner
#INTJ: interjection
#NOUN: noun
#NUM: numeral
#PART: particle
#PRON: pronoun
#PROPN: proper noun
#PUNCT: punctuation
#SCONJ: subordinating conjunction
#SYM: symbol
#VERB: verb
#X: other

upos_tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 
             'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']

nlp = spacy.load("de_core_news_sm")


def pos_features(text):
    doc = nlp(text)
    doc_pos = [token.pos_ for token in doc]
    doc_length = len(doc_pos)
    counts = {tag: doc_pos.count(tag)/doc_length for tag in upos_tags} # proportion of pos tags in text
    return counts


def entity_feature(text):
    doc = nlp(text)
    doc_ent = [token.ent_iob_ for token in doc]
    doc_length = len(doc_ent)
    count = (doc_ent.count('B')+doc_ent.count('I'))/doc_length # proportion of entities in text
    return count


def morph_features(text):
    doc = nlp(text)
    doc_morph = [token.morph for token in doc]
    doc_length = len(doc_morph)
    tense = sum([1 for token in doc_morph if "Tense=Pres" in token])/doc_length
    mood = sum([1 for token in doc_morph if "Mood=Imp" in token])/doc_length
    person = sum([1 for token in doc_morph if "Person=1" in token])/doc_length
    return {"pres_tense": tense, "imperative": mood, "first_person": person}

In [20]:
# add column with pos tags (as dict)
corpus_de['POS'] = corpus_de['argument'].apply(lambda x: pos_features(x))
# pos dict to single columns
corpus_de = pd.concat([corpus_de, corpus_de['POS'].apply(pd.Series)], axis=1)
corpus_de = corpus_de.drop('POS', axis=1)

In [21]:
# add column with entity ratio
corpus_de['Entities'] = corpus_de['argument'].apply(lambda x: entity_feature(x))

In [22]:
# add column with morphology features
corpus_de['Morph'] = corpus_de['argument'].apply(lambda x: morph_features(x))
# morph dict to single columns
corpus_de = pd.concat([corpus_de, corpus_de['Morph'].apply(pd.Series)], axis=1)
corpus_de = corpus_de.drop('Morph', axis=1)

In [23]:
corpus_de.head()

,argument_id,argument,stance,topic,gender,age,residence,civil_status,denomination,education,...,PROPN,PUNCT,SCONJ,SYM,VERB,X,Entities,pres_tense,imperative,first_person
0,201900,Das Schweizer Volk hat die MEI angenommen und ...,FAVOR,Immigration,Männlich,18-34,Land,Ledig,Christ-katholisch,Fachhochschule,...,0.03125,0.093750,0.000000,0.0,0.125000,0.0,0.09375,0.062500,0.0,0.000000
1,201901,Eine Legalisierung von Cannabis entlasten die ...,FAVOR,Society,Männlich,18-34,Land,Ledig,Christ-katholisch,Fachhochschule,...,0.00000,0.133333,0.033333,0.0,0.166667,0.0,0.00000,0.166667,0.0,0.000000
33,2019054,Die jetzige finanzielle Situation der Pensions...,FAVOR,Welfare,Männlich,50-64,Land,Verheiratet,Evangelischreformiert/protestantisch,Höhere Berufsausbildung,...,0.00000,0.083333,0.000000,0.0,0.083333,0.0,0.00000,0.083333,0.0,0.083333
34,2019055,"Dort wo die Leistungen nicht angebracht sind, ...",FAVOR,Welfare,Männlich,50-64,Land,Verheiratet,Evangelischreformiert/protestantisch,Höhere Berufsausbildung,...,0.00000,0.166667,0.083333,0.0,0.083333,0.0,0.00000,0.083333,0.0,0.166667
35,2019056,"Wünschenswert, aber eine Herkulesaufgabe für d...",FAVOR,Education,Männlich,50-64,Land,Verheiratet,Evangelischreformiert/protestantisch,Höhere Berufsausbildung,...,0.06250,0.187500,0.000000,0.0,0.062500,0.0,0.06250,0.062500,0.0,0.000000


#### Mean and standard deviation for the features

In [24]:
syntactic_features = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 
                    'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'Entities', 'pres_tense',
                    'imperative', 'first_person']

In [25]:
for feat in syntactic_features:
    print(feat, round(corpus_de[feat].mean(),2), round(corpus_de[feat].min(),2), 
          round(corpus_de[feat].max(),2), round(corpus_de[feat].std(),2))

ADJ 0.06 0.0 0.43 0.05
ADP 0.08 0.0 0.43 0.05
ADV 0.11 0.0 0.62 0.07
AUX 0.08 0.0 0.38 0.05
CCONJ 0.03 0.0 0.23 0.03
DET 0.12 0.0 0.36 0.06
INTJ 0.0 0.0 0.0 0.0
NOUN 0.21 0.0 0.8 0.07
NUM 0.0 0.0 0.25 0.02
PART 0.02 0.0 0.27 0.03
PRON 0.05 0.0 0.4 0.05
PROPN 0.01 0.0 0.38 0.03
PUNCT 0.12 0.0 0.5 0.05
SCONJ 0.01 0.0 0.17 0.02
SYM 0.0 0.0 0.0 0.0
VERB 0.09 0.0 0.5 0.05
X 0.0 0.0 0.22 0.01
Entities 0.04 0.0 0.62 0.05
pres_tense 0.08 0.0 0.3 0.04
imperative 0.0 0.0 0.11 0.0
first_person 0.02 0.0 0.5 0.05


#### Save the prepared corpus for OLS regression

The OLS regression is conducted in the R script.

In [26]:
corpus_de.shape

(11289, 43)

In [27]:
corpus_de.to_csv('corpus_de_ols_1_nondummy_nomv.csv', sep="\t", index=False)